In [1]:
import ibis

con = ibis.connect("duckdb://penguins.ddb")

ibis.options.interactive = True

# Create table

In [2]:
con.create_table(
    "penguins", ibis.examples.penguins.fetch().to_pyarrow(), overwrite=True
)

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year  ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ int64 │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │  2007 │
│ Adelie  │ Torgersen │           39.5 │          17.4 │               186 │        3800 │ female │  2007 │
│ Adelie  │ Torgersen │           40.3 │          18.0 │               195 │        3250 │ female │  2007 │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL   │  2007 │
│ Adelie  │ Torgersen │           36.7 │          19.3 │               193 │        3450 │ female │  2007 │
│ Adelie  │ Torgersen │           39.3 │          20.6 │               190 │        3650 │ male   │  2007 │
│ Adelie  │ Torgersen │           38.9 │          17.8 │               181 │        3625 │ female │  2007 │
│ Adelie  │ Torgersen │           39.2 │          19.6 │               195 │        4675 │ male   │  2007 │
│ Adelie  │ Torgersen │           34.1 │          18.1 │               193 │        3475 │ NULL   │  2007 │
│ Adelie  │ Torgersen │           42.0 │          20.2 │               190 │        4250 │ NULL   │  2007 │
│ …       │ …         │              … │             … │                 … │           … │ …      │     … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────┘

# Load table

In [3]:
penguins = con.table("penguins")

In [4]:
penguins_pdf = penguins.execute()

# Filter

In [ ]:
penguins.filter(penguins.species == "Adelie")

In [ ]:
penguins_pdf.loc[penguins_pdf.species == "Adelie"].head()

In [ ]:
penguins.filter((penguins.island == "Torgersen") & (penguins.species == "Adelie"))

In [ ]:
penguins_pdf.loc[
    (penguins_pdf.island == "Torgersen") & (penguins_pdf.species == "Adelie")
].head()

# Select

In [ ]:
penguins.select("species", "island", "year")

In [ ]:
penguins_pdf[["species", "island", "year"]].head()

# Mutate

In [ ]:
penguins.mutate(
    bill_length_cm=penguins.bill_length_mm / 10, continent=ibis.literal("Antarctica")
)

In [ ]:
penguins_pdf.assign(
    bill_length_cm=penguins_pdf.bill_length_mm / 10, continent="Antarctica"
).head()

# Selectors

In [ ]:
import ibis.selectors as s

penguins.mutate(bill_length_cm=penguins.bill_length_mm / 10).select(
    ~s.matches("bill_length_mm")
    # match every column except `bill_length_mm`
)

In [ ]:
penguins_pdf.assign(
    bill_length_cm=penguins_pdf.bill_length_mm / 10,
)[[col for col in penguins_pdf.columns if col != "bill_length_mm"]].head()

In [ ]:
penguins.select("island", s.numeric())

In [ ]:
penguins_pdf.select_dtypes("number").head()

# Order by

In [ ]:
penguins.order_by(penguins.flipper_length_mm).select(
    "species", "island", "flipper_length_mm"
)

In [ ]:
penguins_pdf.sort_values(["flipper_length_mm"])[
    ["species", "island", "flipper_length_mm"]
].head()

# Aggregates

In [ ]:
penguins.flipper_length_mm.mean()

In [ ]:
penguins_pdf.flipper_length_mm.mean()

In [ ]:
penguins.aggregate([penguins.flipper_length_mm.mean(), penguins.bill_depth_mm.max()])

In [ ]:
penguins_pdf.aggregate({"flipper_length_mm": "mean", "bill_depth_mm": "max"})

# Group by

In [ ]:
penguins.group_by(["species", "island"]).aggregate(
    penguins.bill_length_mm.mean().name("mean_bill_length_mm")
)

In [ ]:
penguins_pdf.groupby(["species", "island"]).agg(
    mean_bill_length_mm=("bill_length_mm", "mean")
)

# Compile

In [31]:
long_expr = (
    penguins.filter((penguins.sex == "female") & (penguins.year == 2008))
    .group_by(["island"])
    .aggregate(penguins.body_mass_g.max().name("max_body_mass_g"))
)

In [32]:
long_expr

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ island    ┃ max_body_mass_g ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string    │ int64           │
├───────────┼─────────────────┤
│ Torgersen │            3800 │
│ Dream     │            3900 │
│ Biscoe    │            5200 │
└───────────┴─────────────────┘

In [33]:
long_expr.to_sql(dialect="impala")

```sql
SELECT
  `t1`.`island`,
  MAX(`t1`.`body_mass_g`) AS `max_body_mass_g`
FROM (
  SELECT
    *
  FROM `penguins` AS `t0`
  WHERE
    `t0`.`sex` = 'female' AND `t0`.`year` = 2008
) AS `t1`
GROUP BY
  1
```

In [34]:
long_expr.to_sql(dialect="athena")

```sql
SELECT
  "t1"."island",
  MAX("t1"."body_mass_g") AS "max_body_mass_g"
FROM (
  SELECT
    *
  FROM "penguins" AS "t0"
  WHERE
    "t0"."sex" = 'female' AND "t0"."year" = 2008
) AS "t1"
GROUP BY
  1
```

# Raw SQL

In [ ]:
# penguins table need to exist

penguins.sql(
    """  
SELECT island, mean(bill_length_mm) AS avg_bill_length  
FROM penguins  
GROUP BY 1  
ORDER BY 2 DESC  
"""
)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ island    ┃ avg_bill_length ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string    │ float64         │
├───────────┼─────────────────┤
│ Biscoe    │       45.257485 │
│ Dream     │       44.167742 │
│ Torgersen │       38.950980 │
└───────────┴─────────────────┘

In [38]:
long_expr

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ island    ┃ max_body_mass_g ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string    │ int64           │
├───────────┼─────────────────┤
│ Biscoe    │            5200 │
│ Torgersen │            3800 │
│ Dream     │            3900 │
└───────────┴─────────────────┘

In [40]:
# throws error since long_expr is not a table in the database

# long_expr.sql(
#     """
# SELECT *
# FROM long_expr
# WHERE island = 'Torgersen'
# """
# )

In [45]:
con.sql(
    f"""
SELECT *
FROM ({long_expr.to_sql()}) 
WHERE island = 'Torgersen'
"""
)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ island    ┃ max_body_mass_g ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string    │ int64           │
├───────────┼─────────────────┤
│ Torgersen │            3800 │
└───────────┴─────────────────┘

In [46]:
long_expr.sql(
    f"""
SELECT *
FROM ({long_expr.to_sql()}) 
WHERE island = 'Torgersen'
"""
)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ island    ┃ max_body_mass_g ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string    │ int64           │
├───────────┼─────────────────┤
│ Torgersen │            3800 │
└───────────┴─────────────────┘

In [48]:
con.sql(
    f"""  
SELECT island, mean(bill_length_mm) AS avg_bill_length  
FROM ({penguins.to_sql()})  
GROUP BY 1  
ORDER BY 2 DESC  
"""
)

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ island    ┃ avg_bill_length ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string    │ float64         │
├───────────┼─────────────────┤
│ Biscoe    │       45.257485 │
│ Dream     │       44.167742 │
│ Torgersen │       38.950980 │
└───────────┴─────────────────┘